In [138]:
import pandas as pd
import re
import string
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [139]:
file_name = 'data.csv' 
tweet_data = pd.read_csv(file_name, encoding='latin-1')

### EDA

In [140]:
tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [141]:
tweet_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13169 entries, 0 to 13168
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet          13169 non-null  object
 1   HS             13169 non-null  int64 
 2   Abusive        13169 non-null  int64 
 3   HS_Individual  13169 non-null  int64 
 4   HS_Group       13169 non-null  int64 
 5   HS_Religion    13169 non-null  int64 
 6   HS_Race        13169 non-null  int64 
 7   HS_Physical    13169 non-null  int64 
 8   HS_Gender      13169 non-null  int64 
 9   HS_Other       13169 non-null  int64 
 10  HS_Weak        13169 non-null  int64 
 11  HS_Moderate    13169 non-null  int64 
 12  HS_Strong      13169 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.3+ MB


In [142]:
tweet_data.describe()

,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
count,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000,13169.000000
mean,0.422280,0.382945,0.271471,0.150809,0.060217,0.042980,0.024527,0.023236,0.284000,0.256891,0.129471,0.035918
std,0.493941,0.486123,0.444735,0.357876,0.237898,0.202819,0.154685,0.150659,0.450954,0.436935,0.335733,0.186092
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [143]:
tweet_data.dtypes

Tweet            object
HS                int64
Abusive           int64
HS_Individual     int64
HS_Group          int64
HS_Religion       int64
HS_Race           int64
HS_Physical       int64
HS_Gender         int64
HS_Other          int64
HS_Weak           int64
HS_Moderate       int64
HS_Strong         int64
dtype: object

## DATA CLEANING & PREPROCESSING

In [144]:
Duplicated_row = tweet_data.duplicated().sum()
print ("Jumlah duplikat data terdeteksi:", Duplicated_row)

Jumlah duplikat data terdeteksi: 125


In [145]:
tweet_data_cleaned = tweet_data.drop_duplicates()
print("Jumlah duplikat data setelah dihapus:", tweet_data_cleaned.duplicated().sum())

Jumlah duplikat data setelah dihapus: 0


In [147]:
print(tweet_data.isnull().sum())

Tweet            0
HS               0
Abusive          0
HS_Individual    0
HS_Group         0
HS_Religion      0
HS_Race          0
HS_Physical      0
HS_Gender        0
HS_Other         0
HS_Weak          0
HS_Moderate      0
HS_Strong        0
dtype: int64


In [148]:
tweet_data.to_csv('data.csv', index=False)
print(f"Jumlah baris data setelah pembersihan: {len(tweet_data_cleaned)}")

Jumlah baris data setelah pembersihan: 13044


### CASE FOLDING 

Case folding adalah proses mengonversi semua karakter dalam sebuah teks ke bentuk huruf kecil atau huruf besar yang seragam. 
Tujuannya adalah untuk mengurangi kompleksitas dalam analisis teks dan memperlakukan karakter huruf besar dan huruf kecil sebagai setara. 
Pada penggalan code dibawah digunakan `.str.lower()` untuk mengoversi semua karakter ke huruf kecil.

In [149]:
tweet_data['Tweet_lower'] = tweet_data['Tweet'].str.lower()

tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,Tweet_lower
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,- disaat semua cowok berusaha melacak perhatia...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,rt user: user siapa yang telat ngasih tau elu?...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,"41. kadang aku berfikir, kenapa aku tetap perc..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,user user aku itu aku\n\nku tau matamu sipit t...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,user user kaum cebong kapir udah keliatan dong...


### FILTERING

Filtering merupakan proses untuk membersihkan data dari karakter-karakter yang tidak diperlukan. Pada baris code dibawah terdapat beberapa proses filtering yang dilakukan, yaitu sebagai berikut:
1. Method `remove_tweet_special(text)`: Berfungsi untuk menggantikan beberapa karakter khusus dan mengonversi teks ke format ASCII.
2. Method `remove_punctuation(text)`: Berfungsi untuk menghapus tanda baca dari teks
3. Method `remove_whitespace_LT(text)`: Berfungsi untuk menghapus spasi kosong di awal dan akhir teks.
4. Method `remove_whitespace_multiple(text)`: Berfungsi untuk menggantikan beberapa spasi putih berturut-turut dengan satu spasi.
5. Method `remove_single_char(text)`: Berfungsi untuk menghapus kata-kata yang hanya terdiri dari satu karakter.


In [150]:
def remove_tweet_special (text) :
    text = str(text).replace("\\t", " ").replace("\\n", " ").replace("\\u", " ").replace("\\", "")
    text = str(text).encode('ascii', 'replace').decode('ascii')
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",text).split())
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet'].apply(remove_tweet_special)

def remove_punctuation (text) :
    text = text.translate(str.maketrans("","",string.punctuation))
    return text

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_punctuation)

def remove_whitespace_LT(text):
    return text.strip()

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_whitespace_LT)

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_whitespace_multiple)

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

tweet_data['Tweet_filtering'] = tweet_data['Tweet_filtering'].apply(remove_single_char)

tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,Tweet_lower,Tweet_filtering
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,rt user: user siapa yang telat ngasih tau elu?...,RT USER USER siapa yang telat ngasih tau elued...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,"41. kadang aku berfikir, kenapa aku tetap perc...",41 Kadang aku berfikir kenapa aku tetap percay...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,user user aku itu aku\n\nku tau matamu sipit t...,USER USER AKU ITU AKU KU TAU MATAMU SIPIT TAPI...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,user user kaum cebong kapir udah keliatan dong...,USER USER Kaum cebong kapir udah keliatan dong...


### TOKENIZING

Tokenizing adalah proses memecah teks menjadi unit-unit yang lebih kecil, yang disebut "token". Token adalah unit dasar dalam pemrosesan teks dan dapat berupa kata, frasa, atau karakter, tergantung pada tingkat granularitas yang diinginkan. Pada baris code dibawah method `word_tokenize_wrapper(text)` berfungsi untuk memecah teks menjadi tingkatan token kata. Kemudian kata-kata tersebut disimpan dalam kolom baru bernama **Tweet_token**.

In [151]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)


tweet_data["Tweet_token"] = tweet_data["Tweet_filtering"].apply(word_tokenize_wrapper)

tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,Tweet_lower,Tweet_filtering,Tweet_token
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,"[disaat, semua, cowok, berusaha, melacak, perh..."
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,rt user: user siapa yang telat ngasih tau elu?...,RT USER USER siapa yang telat ngasih tau elued...,"[RT, USER, USER, siapa, yang, telat, ngasih, t..."
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,"41. kadang aku berfikir, kenapa aku tetap perc...",41 Kadang aku berfikir kenapa aku tetap percay...,"[41, Kadang, aku, berfikir, kenapa, aku, tetap..."
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,user user aku itu aku\n\nku tau matamu sipit t...,USER USER AKU ITU AKU KU TAU MATAMU SIPIT TAPI...,"[USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S..."
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,user user kaum cebong kapir udah keliatan dong...,USER USER Kaum cebong kapir udah keliatan dong...,"[USER, USER, Kaum, cebong, kapir, udah, keliat..."


In [152]:
def FreqDist_wrapper(text):
    return FreqDist(text)

tweet_data['Tweet_token_freq'] = tweet_data['Tweet_token'].apply(FreqDist_wrapper)

print('Frequency Distribution :')
print(tweet_data['Tweet_token_freq'].head().apply(lambda x : x.most_common()))

Frequency Distribution :
0    [(cowok, 2), (perhatian, 2), (gue, 2), (elo, 2...
1    [(USER, 2), (siapa, 2), (RT, 1), (yang, 1), (t...
2    [(aku, 6), (Kadang, 2), (Tuhan, 2), (Ketika, 2...
3    [(AKU, 3), (USER, 2), (ITU, 2), (KU, 1), (TAU,...
4    [(USER, 2), (Kaum, 1), (cebong, 1), (kapir, 1)...
Name: Tweet_token_freq, dtype: object


### REMOVE STOPWORD


Penghapusan stopwords adalah tahap dalam pra-pemrosesan teks yang bertujuan untuk menghilangkan kata-kata umum yang sering muncul dalam sebuah teks dan tidak memberikan banyak informasi khusus tentang kontennya. Kata-kata tersebut disebut "stopwords." Contoh stopwords dalam bahasa Indonesia meliputi kata-kata seperti "dan," "atau," "yang," dan sebagainya. Pada baris code dibawah, setelah kata-kata pada dataset diubah menjadi token kata, maka dilakukan perbandingan tiap kata pada dataset dengan kata stopwords indonesia. Kata yang diambil hanya kata yang tidak ada pada stopwords indonesia.

In [153]:
stop_words = stopwords.words('indonesian')

def stopwords_removal(words):
    return [word for word in words if word not in stop_words]

tweet_data['Tweet_stopword'] = tweet_data['Tweet_token'].apply(stopwords_removal)

print('Stopword Removal :')
print(tweet_data['Tweet_stopword'].head())

Stopword Removal :
0    [disaat, cowok, berusaha, melacak, perhatian, ...
1    [RT, USER, USER, telat, ngasih, tau, eluedan, ...
2    [41, Kadang, berfikir, percaya, Tuhan, jatuh, ...
3    [USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...
4    [USER, USER, Kaum, cebong, kapir, udah, keliat...
Name: Tweet_stopword, dtype: object


### NORMALIZATION

Tahapan normalisasi dilakukan dengan melakukan normalisasi singkatan atau kata-kata tertentu dengan menggunakan kamus yang disimpan dalam file Excel (**kamus_singkatan.xlsx**). Normalisasi dapat membantu menggantikan singkatan atau variasi kata dengan bentuk yang lebih standar atau lengkap, mempermudah pemrosesan dan analisis teks.

In [154]:
normalize_word = pd.read_csv('kamus_singkatan.csv', encoding='latin-1')

normalize_word_dict = {}

for index, row in normalize_word.iterrows():
    if row.iloc[0] not in normalize_word_dict:
        normalize_word_dict[row.iloc[0]] = row.iloc[1] 

def normalized_term(document):
    return [normalize_word_dict[term] if term in normalize_word_dict else term for term in document]

tweet_data['Tweet_normalized'] = tweet_data['Tweet_stopword'].apply(normalized_term)

print('Normalized Term :')
print(tweet_data['Tweet_normalized'].head())

Normalized Term :
0    [di saat, cowok, berusaha, melacak, perhatian,...
1    [RT, USER, USER, telat, memberi, tau, eluedan,...
2    [41, Kadang, berpikir, percaya, Tuhan, jatuh, ...
3    [USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...
4    [USER, USER, Kaum, cebong, kafir, sudah, kelih...
Name: Tweet_normalized, dtype: object


### STEMMING

Stemming adalah proses menghilangkan infleksi atau afiksasi dari kata-kata sehingga hanya menyisakan akar kata atau bentuk dasar. Stemming membantu dalam mengatasi variasi kata yang mungkin muncul dalam teks dan mengubahnya menjadi bentuk yang lebih seragam. Pada Baris code dibawah, method `stemmed_wrapper(term)` berfungsi untuk mereduksi kata ke bentuk dasar atau akar kata.

In [155]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in tweet_data['Tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print('Total distinct stemmed term :',len(term_dict))

Total distinct stemmed term : 32938


In [156]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,':',term_dict[term])

def get_stemmed_term(document):
    return [term_dict[term] for term in document]

tweet_data['Tweet_stemmed'] = tweet_data['Tweet_normalized'].apply(get_stemmed_term)

di saat : di saat
cowok : cowok
berusaha : usaha
melacak : lacak
perhatian : perhati
gue : gue
kamu : kamu
lantas : lantas
remehkan : remeh
yang : yang
kasih : kasih
khusus : khusus
basic : basic
bego : bego
RT : rt
USER : user
telat : telat
memberi : beri
tau : tau
eluedan : eluedan
sarap : sarap
bergaul : gaul
cigax : cigax
jifla : jifla
calis : cal
itu : itu
licew : licew
41 : 41
Kadang : kadang
berpikir : pikir
percaya : percaya
Tuhan : tuhan
jatuh : jatuh
berkalikali : berkalikali
meninggalkan : tinggal
Ketika : ketika
orang tuaku : orang tua
berencana : rencana
berpisah : pisah
kakakku : kakak
memilih : pilih
Kristen : kristen
anak : anak
ter : ter
AKU : aku
ITU : itu
KU : ku
TAU : tau
MATAMU : mata
SIPIT : sipit
TAPI : tapi
DILIAT : liat
DARI : dari
MANA : mana
Kaum : kaum
cebong : cebong
kafir : kafir
sudah : sudah
kelihatan : lihat
dongoknya : dongok
dungu : dungu
haha : haha
Ya : ya
bani : bani
taplak : taplak
dan kawan kawan : dan kawan kawan
xf0x9fx98x84xf0x9fx98x84xf0x9fx9

In [157]:
def remove_punc(text):
    text = " ".join([char for char in text if char not in string.punctuation])
    return text

tweet_data['Tweet_stemmed_join'] = tweet_data['Tweet_stemmed'].apply(lambda x: remove_punc(x))

tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,...,HS_Moderate,HS_Strong,Tweet_lower,Tweet_filtering,Tweet_token,Tweet_token_freq,Tweet_stopword,Tweet_normalized,Tweet_stemmed,Tweet_stemmed_join
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,...,0,0,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,"[disaat, semua, cowok, berusaha, melacak, perh...","{'disaat': 1, 'semua': 1, 'cowok': 2, 'berusah...","[disaat, cowok, berusaha, melacak, perhatian, ...","[di saat, cowok, berusaha, melacak, perhatian,...","[di saat, cowok, usaha, lacak, perhati, gue, k...",di saat cowok usaha lacak perhati gue kamu lan...
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,...,0,0,rt user: user siapa yang telat ngasih tau elu?...,RT USER USER siapa yang telat ngasih tau elued...,"[RT, USER, USER, siapa, yang, telat, ngasih, t...","{'RT': 1, 'USER': 2, 'siapa': 2, 'yang': 1, 't...","[RT, USER, USER, telat, ngasih, tau, eluedan, ...","[RT, USER, USER, telat, memberi, tau, eluedan,...","[rt, user, user, telat, beri, tau, eluedan, sa...",rt user user telat beri tau eluedan sarap gue ...
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,...,0,0,"41. kadang aku berfikir, kenapa aku tetap perc...",41 Kadang aku berfikir kenapa aku tetap percay...,"[41, Kadang, aku, berfikir, kenapa, aku, tetap...","{'41': 1, 'Kadang': 2, 'aku': 6, 'berfikir': 1...","[41, Kadang, berfikir, percaya, Tuhan, jatuh, ...","[41, Kadang, berpikir, percaya, Tuhan, jatuh, ...","[41, kadang, pikir, percaya, tuhan, jatuh, ber...",41 kadang pikir percaya tuhan jatuh berkalikal...
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,...,0,0,user user aku itu aku\n\nku tau matamu sipit t...,USER USER AKU ITU AKU KU TAU MATAMU SIPIT TAPI...,"[USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...","{'USER': 2, 'AKU': 3, 'ITU': 2, 'KU': 1, 'TAU'...","[USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...","[USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...","[user, user, aku, itu, aku, ku, tau, mata, sip...",user user aku itu aku ku tau mata sipit tapi l...
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,...,1,0,user user kaum cebong kapir udah keliatan dong...,USER USER Kaum cebong kapir udah keliatan dong...,"[USER, USER, Kaum, cebong, kapir, udah, keliat...","{'USER': 2, 'Kaum': 1, 'cebong': 1, 'kapir': 1...","[USER, USER, Kaum, cebong, kapir, udah, keliat...","[USER, USER, Kaum, cebong, kafir, sudah, kelih...","[user, user, kaum, cebong, kafir, sudah, lihat...",user user kaum cebong kafir sudah lihat dongok...


In [158]:
tweet_data["Tweet" ] = tweet_data["Tweet_stemmed_join"]
tweet_data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,...,HS_Moderate,HS_Strong,Tweet_lower,Tweet_filtering,Tweet_token,Tweet_token_freq,Tweet_stopword,Tweet_normalized,Tweet_stemmed,Tweet_stemmed_join
0,di saat cowok usaha lacak perhati gue kamu lan...,1,1,1,0,0,0,0,0,1,...,0,0,- disaat semua cowok berusaha melacak perhatia...,disaat semua cowok berusaha melacak perhatian ...,"[disaat, semua, cowok, berusaha, melacak, perh...","{'disaat': 1, 'semua': 1, 'cowok': 2, 'berusah...","[disaat, cowok, berusaha, melacak, perhatian, ...","[di saat, cowok, berusaha, melacak, perhatian,...","[di saat, cowok, usaha, lacak, perhati, gue, k...",di saat cowok usaha lacak perhati gue kamu lan...
1,rt user user telat beri tau eluedan sarap gue ...,0,1,0,0,0,0,0,0,0,...,0,0,rt user: user siapa yang telat ngasih tau elu?...,RT USER USER siapa yang telat ngasih tau elued...,"[RT, USER, USER, siapa, yang, telat, ngasih, t...","{'RT': 1, 'USER': 2, 'siapa': 2, 'yang': 1, 't...","[RT, USER, USER, telat, ngasih, tau, eluedan, ...","[RT, USER, USER, telat, memberi, tau, eluedan,...","[rt, user, user, telat, beri, tau, eluedan, sa...",rt user user telat beri tau eluedan sarap gue ...
2,41 kadang pikir percaya tuhan jatuh berkalikal...,0,0,0,0,0,0,0,0,0,...,0,0,"41. kadang aku berfikir, kenapa aku tetap perc...",41 Kadang aku berfikir kenapa aku tetap percay...,"[41, Kadang, aku, berfikir, kenapa, aku, tetap...","{'41': 1, 'Kadang': 2, 'aku': 6, 'berfikir': 1...","[41, Kadang, berfikir, percaya, Tuhan, jatuh, ...","[41, Kadang, berpikir, percaya, Tuhan, jatuh, ...","[41, kadang, pikir, percaya, tuhan, jatuh, ber...",41 kadang pikir percaya tuhan jatuh berkalikal...
3,user user aku itu aku ku tau mata sipit tapi l...,0,0,0,0,0,0,0,0,0,...,0,0,user user aku itu aku\n\nku tau matamu sipit t...,USER USER AKU ITU AKU KU TAU MATAMU SIPIT TAPI...,"[USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...","{'USER': 2, 'AKU': 3, 'ITU': 2, 'KU': 1, 'TAU'...","[USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...","[USER, USER, AKU, ITU, AKU, KU, TAU, MATAMU, S...","[user, user, aku, itu, aku, ku, tau, mata, sip...",user user aku itu aku ku tau mata sipit tapi l...
4,user user kaum cebong kafir sudah lihat dongok...,1,1,0,1,1,0,0,0,0,...,1,0,user user kaum cebong kapir udah keliatan dong...,USER USER Kaum cebong kapir udah keliatan dong...,"[USER, USER, Kaum, cebong, kapir, udah, keliat...","{'USER': 2, 'Kaum': 1, 'cebong': 1, 'kapir': 1...","[USER, USER, Kaum, cebong, kapir, udah, keliat...","[USER, USER, Kaum, cebong, kafir, sudah, kelih...","[user, user, kaum, cebong, kafir, sudah, lihat...",user user kaum cebong kafir sudah lihat dongok...


Setelah dilakukan preprocessing, data selanjutnya dimasukkan ke dalam file csv `preprocessing_Data.csv`

In [159]:
tweet_data.to_csv('preprocessing_data.csv',encoding='utf8', index=False)